In [ ]:
# Migrate Existing Repository to Monorepo Pattern
# This notebook helps you reorganize your existing repository into the recommended structure

from pathlib import Path
import shutil
import subprocess
import json
from datetime import datetime
import os

class RepositoryMigrator:
    """Migrate existing repository to monorepo pattern"""
    
    def __init__(self, repo_path="."):
        self.repo_path = Path(repo_path).resolve()
        self.backup_dir = self.repo_path / "backup_before_migration"
        
    def analyze_current_structure(self):
        """Analyze what's currently in the repository"""
        current_files = []
        current_dirs = []
        
        for item in self.repo_path.iterdir():
            if item.name.startswith('.') or item.name == 'backup_before_migration':
                continue
                
            if item.is_file():
                current_files.append(item.name)
            elif item.is_dir():
                current_dirs.append(item.name)
        
        print("Current Repository Structure:")
        print(f"Files: {current_files}")
        print(f"Directories: {current_dirs}")
        
        return current_files, current_dirs
    
    def create_backup(self):
        """Create backup of current state"""
        self.backup_dir.mkdir(exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_path = self.backup_dir / f"backup_{timestamp}"
        backup_path.mkdir()
        
        for item in self.repo_path.iterdir():
            if item.name not in ['.git', 'backup_before_migration']:
                if item.is_file():
                    shutil.copy2(item, backup_path)
                elif item.is_dir():
                    shutil.copytree(item, backup_path / item.name)
        
        print(f"✅ Backup created at: {backup_path}")
        return backup_path
    
    def create_monorepo_structure(self):
        """Create the recommended monorepo directory structure"""
        structure = {
            "projects": [
                "01-mcp-basics",
                "02-github-integration", 
                "03-travel-assistant"
            ],
            "docs": [
                "learning-notes",
                "progress",
                "references"
            ],
            "experiments": [],
            "resources": [],
            "scripts": []
        }
        
        for main_dir, sub_dirs in structure.items():
            main_path = self.repo_path / main_dir
            main_path.mkdir(exist_ok=True)
            
            for sub_dir in sub_dirs:
                if sub_dir:  # Skip empty string
                    sub_path = main_path / sub_dir
                    sub_path.mkdir(exist_ok=True)
                    
                    # Create placeholder README
                    self.create_placeholder_readme(sub_path, sub_dir)
    
    def create_placeholder_readme(self, path, name):
        """Create a README file for each directory"""
        readme_content = f"""# {name.replace('-', ' ').title()}

## Overview
This directory is part of the Git and MCP learning journey.

## Contents
[Add description of contents here]

## Last Updated
{datetime.now().strftime('%Y-%m-%d')}
"""
        with open(path / "README.md", "w") as f:
            f.write(readme_content)
    
    def migrate_existing_files(self, mapping_strategy='auto'):
        """Migrate existing files to new structure"""
        # Define mapping patterns
        migration_map = {
            # File patterns -> target directory
            'mcp_*': 'projects/01-mcp-basics',
            '*github*': 'projects/02-github-integration',
            '*travel*': 'projects/03-travel-assistant',
            '*.md': 'docs',
            '*.py': 'scripts',
            '*test*': 'experiments',
            '*example*': 'experiments'
        }
        
        if mapping_strategy == 'auto':
            moved_files = []
            for pattern, target_dir in migration_map.items():
                for file_path in self.repo_path.glob(pattern):
                    if file_path.is_file() and not self._is_in_structure(file_path):
                        self._move_with_git(file_path, self.repo_path / target_dir)
                        moved_files.append((file_path.name, target_dir))
            
            return moved_files
        
        # If manual mapping is needed, implement here
        return []
    
    def _is_in_structure(self, file_path):
        """Check if file is already in the new structure"""
        structure_dirs = ['projects', 'docs', 'experiments', 'resources', 'scripts']
        return any(part in structure_dirs for part in file_path.parts)
    
    def _move_with_git(self, source, target_dir):
        """Move file using git mv to preserve history"""
        try:
            # Ensure target directory exists
            target_dir.mkdir(parents=True, exist_ok=True)
            
            # Use git mv
            cmd = ['git', 'mv', str(source), str(target_dir / source.name)]
            subprocess.run(cmd, cwd=self.repo_path, check=True)
            print(f"Moved {source.name} to {target_dir}")
        except subprocess.CalledProcessError as e:
            print(f"Warning: Could not move {source.name} with git: {e}")
            # Fallback to regular move
            shutil.move(str(source), str(target_dir / source.name))
    
    def update_main_readme(self):
        """Create or update main README with new structure"""
        readme_content = """# Learn Git and MCP

## Overview
This repository documents my learning journey with Git version control and Model Context Protocol (MCP).

## Recent Update
Migrated to monorepo structure for better organization.

## Repository Structure
```
learn_git_and_mcp/
├── projects/           # Learning projects organized by topic
│   ├── 01-mcp-basics/
│   ├── 02-github-integration/
│   └── 03-travel-assistant/
├── docs/              # Documentation and learning notes
│   ├── learning-notes/
│   ├── progress/
│   └── references/
├── experiments/       # Quick tests and experiments
├── resources/         # Reference materials
└── scripts/           # Utility scripts
```

## Projects
1. **MCP Basics** - Understanding MCP fundamentals
2. **GitHub Integration** - Using MCP with GitHub APIs  
3. **Travel Assistant** - Advanced MCP application development

## Quick Navigation
- [Learning Progress](docs/progress/README.md)
- [Project Index](docs/references/project-index.md)
- [Git Best Practices](docs/references/git-best-practices.md)

## Getting Started
1. Explore projects in numerical order
2. Check docs/progress for learning milestones
3. See experiments/ for quick tests and proof-of-concepts

## Migration Notes
This repository was recently migrated to a monorepo structure for better organization and learning efficiency.
"""
        with open(self.repo_path / "README.md", "w") as f:
            f.write(readme_content)
    
    def create_migration_commit(self, moved_files):
        """Create a commit documenting the migration"""
        # Add all changes
        subprocess.run(['git', 'add', '.'], cwd=self.repo_path)
        
        # Create detailed commit message
        commit_message = f"""Migrate to monorepo structure

- Reorganize repository into projects/docs/experiments structure
- Preserve file history using git mv
- Update README with new organization
- Add placeholder documentation

Files moved:
"""
        for file, target in moved_files:
            commit_message += f"- {file} -> {target}\n"
        
        commit_message += f"\nCreated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        
        try:
            subprocess.run(['git', 'commit', '-m', commit_message], cwd=self.repo_path, check=True)
            # Tag the migration
            subprocess.run(['git', 'tag', '-a', 'monorepo-migration', '-m', 'Migrated to monorepo structure'], 
                         cwd=self.repo_path, check=True)
            print("\n✅ Migration committed and tagged!")
        except subprocess.CalledProcessError as e:
            print(f"Error creating commit: {e}")
    
    def generate_migration_report(self, moved_files):
        """Generate a report of the migration"""
        report_content = f"""# Repository Migration Report

## Migration Date
{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Changes Made

### Structure Created
- `projects/` - For organized learning projects
- `docs/` - For documentation and notes
- `experiments/` - For quick tests
- `resources/` - For reference materials
- `scripts/` - For utility scripts

### Files Moved
"""
        for file, target in moved_files:
            report_content += f"- `{file}` → `{target}/`\n"
        
        report_content += """
### Next Steps
1. Review the new structure
2. Update any broken references
3. Continue your learning journey!

## Backup
A backup of the original structure was created in `backup_before_migration/`
"""
        
        report_path = self.repo_path / "MIGRATION_REPORT.md"
        with open(report_path, "w") as f:
            f.write(report_content)
        
        return report_path

# Interactive migration process
def run_migration():
    """Run the interactive migration process"""
    import sys
    
    print("🚀 Repository Migration to Monorepo Structure")
    print("=" * 50)
    
    migrator = RepositoryMigrator()
    
    # Step 1: Analyze current structure
    print("\n1. Analyzing current repository structure...")
    current_files, current_dirs = migrator.analyze_current_structure()
    
    # Step 2: Confirm proceeding
    proceed = input("\nContinue with migration? (y/n): ").lower()
    if proceed != 'y':
        print("Migration cancelled.")
        return
    
    # Step 3: Create backup
    print("\n2. Creating backup...")
    backup_path = migrator.create_backup()
    
    # Step 4: Create new structure
    print("\n3. Creating monorepo structure...")
    migrator.create_monorepo_structure()
    
    # Step 5: Migrate files
    print("\n4. Migrating existing files...")
    moved_files = migrator.migrate_existing_files()
    
    # Step 6: Update README
    print("\n5. Updating main README...")
    migrator.update_main_readme()
    
    # Step 7: Commit changes
    print("\n6. Committing migration...")
    migrator.create_migration_commit(moved_files)
    
    # Step 8: Generate report
    print("\n7. Generating migration report...")
    report_path = migrator.generate_migration_report(moved_files)
    
    print(f"\n✨ Migration complete! Check {report_path} for details.")
    print("\nNext steps:")
    print("1. Review the new structure")
    print("2. Update any broken references")
    print("3. Push changes to remote: git push origin main --tags")
    print("4. Continue your Git and MCP learning journey!")

# Run the migration
run_migration()

# Optional: Manual file organization helper
def manual_organization_helper():
    """Helper function for manual file organization"""
    print("\n🔍 Manual Organization Helper")
    print("=" * 30)
    
    repo_path = Path.cwd()
    
    # List unorganized files
    unorganized = []
    for item in repo_path.iterdir():
        if item.is_file() and item.name not in ['.gitignore', 'README.md']:
            if not any(part in ['projects', 'docs', 'experiments', 'resources', 'scripts'] 
                      for part in item.parts):
                unorganized.append(item)
    
    if unorganized:
        print("\nUnorganized files found:")
        for i, file in enumerate(unorganized):
            print(f"{i+1}. {file.name}")
        
        print("\nRecommended directories:")
        print("1. projects/01-mcp-basics/")
        print("2. projects/02-github-integration/")
        print("3. projects/03-travel-assistant/")
        print("4. docs/learning-notes/")
        print("5. experiments/")
        print("6. resources/")
        print("7. scripts/")
        
        # Interactive organization
        for file in unorganized:
            choice = input(f"\nMove {file.name} to (1-7, or 's' to skip): ")
            if choice.isdigit() and 1 <= int(choice) <= 7:
                destinations = [
                    "projects/01-mcp-basics/",
                    "projects/02-github-integration/",
                    "projects/03-travel-assistant/",
                    "docs/learning-notes/",
                    "experiments/",
                    "resources/",
                    "scripts/"
                ]
                dest = destinations[int(choice)-1]
                target = repo_path / dest
                target.mkdir(parents=True, exist_ok=True)
                
                # Move with git
                try:
                    subprocess.run(['git', 'mv', str(file), str(target)], check=True)
                    print(f"✅ Moved {file.name} to {dest}")
                except:
                    print(f"❌ Failed to move {file.name}")
    else:
        print("\n✅ All files are already organized!")

# Uncomment to run manual helper
# manual_organization_helper()

🚀 Repository Migration to Monorepo Structure

1. Analyzing current repository structure...
Current Repository Structure:
Files: ['README.md', 'migrate-to-monorepo.ipynb.ipynb']
Directories: ['projects', 'learning_notes', 'mcp_fundamentals', 'git_examples']
